The idea is to focus on just the generative portion of the model and test new models as they are available. We will start by testing Marco o1

In [ ]:
import json
from typing import Any, Dict, List, Optional, Tuple
import requests

In [ ]:
def extract_examples(data: List[Dict], index: int) -> Tuple[str, List[Tuple[str, str]]]:
    """
    Extract the base finding and reference examples from the evaluation results.
    
    Args:
        data: List of dictionaries containing the evaluation results
        index: Index of the item to extract
        
    Returns:
        Tuple containing:
            - Base finding string
            - List of tuples, each containing (finding, conclusion) pairs from reference examples
    """
    item = data[index]
    base_finding = item['finding']
    
    reference_examples = [
        (example['finding'], example['conclusion'])
        for example in item['reference_examples']
    ]
    
    return base_finding, reference_examples

def create_prompt(
    base_finding: str,
    reference_examples: List[Tuple[str, str]],
    base_prompt: Optional[str] = None
) -> str:
    """
    Create a formatted prompt combining the base finding and reference examples.
    
    Args:
        base_finding: The primary finding to generate conclusions for
        reference_examples: List of (finding, conclusion) pairs to use as examples
        base_prompt: Optional custom instructions for the prompt
        
    Returns:
        Formatted prompt string
    """
    if base_prompt is None:
        base_prompt = """Please generate the "Conclusions" section of a medical report based on the provided "Findings" section. 
Below are several example pairs of "Findings" and their corresponding "Conclusions" to help guide the format and style of your response.

After reviewing these examples, please generate an appropriate conclusion for the new finding provided at the end."""

    # Format reference examples
    examples_text = "\n\n".join([
        f"Example {i+1}:\n"
        f"Findings:\n{finding}\n\n"
        f"Conclusion:\n{conclusion}"
        for i, (finding, conclusion) in enumerate(reference_examples)
    ])
    
    # Combine all parts
    full_prompt = f"""{base_prompt}

{examples_text}

New Finding to generate conclusion for:
{base_finding}"""
    
    return full_prompt

In [ ]:
with open("evaluation_results.json", "r") as f:
    data = json.load(f)

In [ ]:
base_finding, reference_examples = extract_examples(data, 0)

In [ ]:
base_finding

In [ ]:
prompt = create_prompt(base_finding, reference_examples)
print(prompt)

In [ ]:
def run_ollama_inference(
    prompt: str,
    model: str = "mistral",
    system_prompt: Optional[str] = None,
    temperature: float = 0.7,
    max_tokens: Optional[int] = None
) -> str:
    """
    Run inference using an Ollama model.
    
    Args:
        prompt: The input text to process
        model: Name of the Ollama model to use (default: "mistral")
        system_prompt: Optional system prompt to set context
        temperature: Sampling temperature (default: 0.7)
        max_tokens: Maximum tokens to generate (optional)
        
    Returns:
        Generated text response
        
    Raises:
        requests.exceptions.RequestException: If the API call fails
    """
    url = "http://localhost:11434/api/generate"
    
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False,
        "temperature": temperature
    }
    
    if system_prompt:
        payload["system"] = system_prompt
    if max_tokens:
        payload["max_tokens"] = max_tokens

    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        return response.json()["response"]
    except requests.exceptions.RequestException as e:
        raise Exception(f"Failed to get response from Ollama: {str(e)}")

In [ ]:
!ollama list

In [ ]:
system_prompt = "You are an AI radiology assistant, helping to generate conclusions from findings in radiology reports. Please make sure that all output is in english."


In [ ]:
response = run_ollama_inference(prompt=prompt, model='marco-o1', system_prompt=system_prompt)

In [ ]:
print(response)

In [ ]:
# Alternative implementation using the ollama package
def run_ollama_client(
    prompt: str,
    model: str = "mistral",
    system_prompt: Optional[str] = None,
    temperature: float = 0.7
) -> str:
    """
    Run inference using the Ollama client package.
    
    Args:
        prompt: The input text to process
        model: Name of the Ollama model to use (default: "mistral")
        system_prompt: Optional system prompt to set context
        temperature: Sampling temperature (default: 0.7)
        
    Returns:
        Generated text response
    """
    try:
        import ollama
    except ImportError:
        raise ImportError("Please install ollama package: pip install ollama")
    
    try:
        response = ollama.generate(
            model=model,
            prompt=prompt,
            system=system_prompt if system_prompt else None,
            temperature=temperature
        )
        return response['response']
    except Exception as e:
        raise Exception(f"Failed to get response from Ollama: {str(e)}")